In [1]:
import json
import zipfile
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [2]:
from pathlib import Path
data_dir = Path('.')
data_dir.mkdir(exist_ok = True)
file_path = data_dir / Path('CVR1.zip')
dest_path = file_path

In [3]:
zip_1 = zipfile.ZipFile(dest_path, 'r')
with zip_1.open('CvrExport.json') as cvr_1:
    cvr_exp = pd.read_json(cvr_1)

In [4]:
cvr_exp.head()

,Version,ElectionId,Sessions
0,5.5.12.1,Bartow Nov 2020 General,"{'TabulatorId': 530, 'BatchId': 1, 'RecordId':..."
1,5.5.12.1,Bartow Nov 2020 General,"{'TabulatorId': 530, 'BatchId': 1, 'RecordId':..."
2,5.5.12.1,Bartow Nov 2020 General,"{'TabulatorId': 530, 'BatchId': 1, 'RecordId':..."
3,5.5.12.1,Bartow Nov 2020 General,"{'TabulatorId': 530, 'BatchId': 1, 'RecordId':..."
4,5.5.12.1,Bartow Nov 2020 General,"{'TabulatorId': 530, 'BatchId': 1, 'RecordId':..."


In [5]:
cvr_exp.size

152034

In [53]:
cvr_sessions = cvr_exp['Sessions']
unit_dict = cvr_sessions[0]
cvr_sessions.head()

0    {'TabulatorId': 530, 'BatchId': 1, 'RecordId':...
1    {'TabulatorId': 530, 'BatchId': 1, 'RecordId':...
2    {'TabulatorId': 530, 'BatchId': 1, 'RecordId':...
3    {'TabulatorId': 530, 'BatchId': 1, 'RecordId':...
4    {'TabulatorId': 530, 'BatchId': 1, 'RecordId':...
Name: Sessions, dtype: object

In [54]:
##Sanity check to make sure cvr_sessions is a series of nested dictionaries
isinstance(unit_dict, dict)

True

In [55]:
##Sorting through to the list of contests, checking type is list for each contest
isinstance(unit_dict['Original']['Cards'][0]['Contests'], list)

True

In [67]:
##Add first level of dict keys/values
unit_df_first = pd.DataFrame.from_dict(unit_dict, orient='index').drop('ImageMask').drop('VotingSessionIdentifier').drop('UniqueVotingIdentifier').drop('Original')
unit_df_first = unit_df_first.T
##Add second level of dict keys/values
unit_df_second = pd.DataFrame.from_dict(unit_dict['Original'], orient='index').drop('Cards')
unit_df_second = unit_df_second.T
##concat both
unit_df_2lvl = pd.concat([unit_df_first, unit_df_second], axis=1, join="inner")
unit_df_2lvl
##third level
unit_df_third = pd.DataFrame.from_dict(unit_dict['Original']['Cards'][0], orient='index').drop('Contests')
unit_df_third = unit_df_third.T
##concat all but contests
unit_df_nocontests = pd.concat([unit_df_2lvl, unit_df_third], axis=1, join="inner")
unit_df_nocontests

,TabulatorId,BatchId,RecordId,CountingGroupId,SessionType,PrecinctPortionId,BallotTypeId,IsCurrent,Id,PaperIndex,OutstackConditionIds
0,530,1,23,3,ScannedVote,4,4,True,21,0,[]


In [78]:
## Contest list to table in unit example
unit_contests = unit_dict['Original']['Cards'][0]['Contests']
unit_contests

[{'Id': 1,
  'ManifestationId': 133,
  'Undervotes': 0,
  'Overvotes': 0,
  'OutstackConditionIds': [],
  'Marks': [{'CandidateId': 2,
    'ManifestationId': 766,
    'PartyId': 0,
    'Rank': 1,
    'MarkDensity': 100,
    'IsAmbiguous': False,
    'IsVote': True,
    'OutstackConditionIds': []}]},
 {'Id': 2,
  'ManifestationId': 134,
  'Undervotes': 0,
  'Overvotes': 0,
  'OutstackConditionIds': [],
  'Marks': [{'CandidateId': 5,
    'ManifestationId': 770,
    'PartyId': 0,
    'Rank': 1,
    'MarkDensity': 98,
    'IsAmbiguous': False,
    'IsVote': True,
    'OutstackConditionIds': []}]},
 {'Id': 3,
  'ManifestationId': 135,
  'Undervotes': 0,
  'Overvotes': 0,
  'OutstackConditionIds': [],
  'Marks': [{'CandidateId': 17,
    'ManifestationId': 783,
    'PartyId': 0,
    'Rank': 1,
    'MarkDensity': 100,
    'IsAmbiguous': False,
    'IsVote': True,
    'OutstackConditionIds': []}]},
 {'Id': 4,
  'ManifestationId': 136,
  'Undervotes': 0,
  'Overvotes': 0,
  'OutstackConditionIds

In [84]:
one_contest = unit_contests[0]
one_contest

{'Id': 1,
 'ManifestationId': 133,
 'Undervotes': 0,
 'Overvotes': 0,
 'OutstackConditionIds': [],
 'Marks': [{'CandidateId': 2,
   'ManifestationId': 766,
   'PartyId': 0,
   'Rank': 1,
   'MarkDensity': 100,
   'IsAmbiguous': False,
   'IsVote': True,
   'OutstackConditionIds': []}]}

In [92]:
one_contest_df = pd.DataFrame.from_dict(one_contest, orient='index').drop('Marks')
one_contest_df = one_contest_df.T
one_contest_df

,Id,ManifestationId,Undervotes,Overvotes,OutstackConditionIds
0,1,133,0,0,[]


In [101]:
one_marks = one_contest['Marks'][0]
one_marks_df = pd.DataFrame.from_dict(one_marks, orient='index').T
one_contest_concat = pd.concat([one_contest_df, one_marks_df], axis=1, join="inner")
one_contest_concat

,Id,ManifestationId,Undervotes,Overvotes,OutstackConditionIds,CandidateId,ManifestationId,PartyId,Rank,MarkDensity,IsAmbiguous,IsVote,OutstackConditionIds
0,1,133,0,0,[],2,766,0,1,100,False,True,[]


In [134]:
## Iterate through all contests for one ballot
unit_contests_df = pd.DataFrame()
for i in range(len(unit_contests)-1):
    contest_df = pd.DataFrame.from_dict(unit_contests[i], orient='index').drop('Marks').T
    if not unit_contests[i]['Marks']:
        for col in marks_df.columns:
            marks_df[col].values[:] = 0
    else:
        marks_df = pd.DataFrame.from_dict(unit_contests[i]['Marks'][0], orient='index').T
    contest_concat = pd.concat([contest_df, marks_df], axis=1, join="inner")
    unit_contests_df = unit_contests_df.append(contest_concat)
unit_contests_df

,Id,ManifestationId,Undervotes,Overvotes,OutstackConditionIds,CandidateId,ManifestationId,PartyId,Rank,MarkDensity,IsAmbiguous,IsVote,OutstackConditionIds
0,1,133,0,0,[],2,766,0,1,100,False,True,[]
0,2,134,0,0,[],5,770,0,1,98,False,True,[]
0,3,135,0,0,[],17,783,0,1,100,False,True,[]
0,4,136,0,0,[],28,795,0,1,98,False,True,[]
0,5,137,0,0,[],31,799,0,1,99,False,True,[]
0,6,138,0,0,[],34,803,0,1,96,False,True,[]
0,7,139,0,0,[],36,806,0,1,98,False,True,[]
0,9,140,1,0,"[4, 6]",0,0,0,0,0,0,0,0
0,12,141,1,0,"[4, 6]",0,0,0,0,0,0,0,0
0,13,142,1,0,"[4, 6]",0,0,0,0,0,0,0,0


[]